In [1]:
import pandas as pd
import numpy as np
import datetime

df = pd.read_csv("../Datasets/Plane Crashes.csv")

In [2]:
df["time_2"] = df["Time"].fillna("00:00")
df["datetime"] =  pd.to_datetime(df['Date'].astype(str) + ' ' + df['time_2'].astype(str))
df.drop(columns=['time_2'],inplace=True)
df.head(2)

,Date,Time,Aircraft,Operator,Registration,Flight phase,Flight type,Survivors,Crash site,Schedule,...,Region,Crew on board,Crew fatalities,Pax on board,PAX fatalities,Other fatalities,Total fatalities,Circumstances,Crash cause,datetime
0,1918-05-02,NaN,De Havilland DH.4,United States Signal Corps - USSC,AS-32084,Takeoff (climb),Test,No,Airport (less than 10 km from airport),Dayton - Dayton,...,North America,2.0,2.0,0.0,0.0,0.0,2,The single engine airplane departed Dayton-McC...,Technical failure,1918-05-02
1,1918-06-08,NaN,Handley Page V/1500,Handley Page Aircraft Company Ltd,E4104,Takeoff (climb),Test,Yes,Airport (less than 10 km from airport),Cricklewood - Cricklewood,...,Europe,6.0,5.0,0.0,0.0,0.0,5,"Assembled at Cricklewood Airfield in May 1918,...",Technical failure,1918-06-08


In [3]:
df["Survivors"] = df["Crew on board"] + df["Pax on board"] - df["Crew fatalities"] - df["PAX fatalities"]

In [4]:
flight_types_mapping = {
    "Test": "testing/training",
    "Training": "testing/training",
    "Calibration": "testing/training",
    "Delivery": "delivery",
    "Postal (mail)": "delivery",
    "Supply": "delivery",
    "Cargo": "delivery",
    "Fire fighting": "support",
    "Refuelling": "support",
    "Spraying (Agricultural)": "support",
    "Ambulance": "support",
    "Ferry": "support",
    "Topographic": "support",
    "Humanitarian": "support",
    "Calibration": "support",
    "Meteorological / Weather": "support",
    "Geographical / Geophysical / Scientific": "support",
    "Aerial photography": "private/business",
    "Scheduled Revenue Flight": "private/business",
    "Private": "private/business",
    "Charter/Taxi (Non Scheduled Revenue Flight)": "private/business",
    "Demonstration": "private/business",
    "Executive/Corporate/Business": "private/business",
    "Cinematography": "private/business",
    "Skydiving / Paratroopers": "private/business",
    "Aerobatic": "private/business",
    "Positioning": "private/business",
    "Military": "government",
    "Government": "government",
    "Survey / Patrol / Reconnaissance": "government",
    "Illegal (smuggling)": "illegal",
    "Bombing": "illegal"
}

In [5]:
df.dropna(subset=["Flight type"], inplace=True)
df["flight_group"] = df["Flight type"].map(flight_types_mapping)

In [6]:
df_crash_cause = pd.DataFrame()
for group in df["flight_group"].unique():
    df_tmp = pd.DataFrame()
    df_tmp["group"] = [group]
    df_tmp["number of crashes"] = [len(df[df["flight_group"] == group])]
    for crash_cause in df["Crash cause"].unique():
        df_tmp[crash_cause] = [len(df[(df["Crash cause"] == crash_cause) & (df["flight_group"] == group)])]
    df_crash_cause = pd.concat([df_crash_cause,df_tmp])

In [7]:
df_crash_cause

,group,number of crashes,Technical failure,Unknown,Weather,Human factor,Other causes,"Terrorism act, Hijacking, Sabotage"
0,testing/training,3865,836,1457,105,1399,51,17
0,government,5791,1188,2748,337,961,82,475
0,delivery,3915,1011,949,259,1366,152,178
0,support,1649,353,438,78,713,43,24
0,private/business,12362,2712,3141,739,5285,293,192
0,illegal,897,101,213,10,191,6,376


In [10]:
df_survivors = pd.DataFrame()
for group in df["flight_group"].unique():
    df_tmp = pd.DataFrame()
    df_tmp["group"] = [group]
    df_tmp["number of crashes"] = [len(df[df["flight_group"] == group])]
    df_tmp["total on-board"] = df[df["flight_group"] == group]["Crew on board"].sum() + df[df["flight_group"] == group]["Pax on board"].sum()
    df_tmp["number of survivors"] = df[df["flight_group"] == group]["Survivors"].sum()
    df_tmp["number of cauelties"] = df[df["flight_group"] == group]["Crew fatalities"].sum() + df[df["flight_group"] == group]["PAX fatalities"].sum() + df[df["flight_group"] == group]["Other fatalities"].sum()
    df_tmp["percentage_survivors"] = df_tmp["number of survivors"]*100/df_tmp["total on-board"]
    df_survivors = pd.concat([df_survivors,df_tmp])

In [11]:
df_survivors

,group,number of crashes,total on-board,number of survivors,number of cauelties,percentage_survivors
0,testing/training,3865,15763.0,5967.0,10293.0,37.854469
0,government,5791,47323.0,13951.0,33796.0,29.480380
0,delivery,3915,13430.0,6379.0,8105.0,47.498138
0,support,1649,6503.0,2469.0,4118.0,37.967092
0,private/business,12362,218292.0,120196.0,98989.0,55.062027
0,illegal,897,5167.0,1857.0,3380.0,35.939617
